In [1]:
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

import os
os.chdir(project_root)

In [2]:
import torch
from torch.utils.data import DataLoader

from src.data.pipeline import get_datasets
from src.models.transformer_model import StockTransformer
from src.training.trainer import Trainer
from src.utils.config import load_config

config = load_config()

train_dataset, val_dataset, test_dataset, feature_columns = get_datasets(config)

train_loader = DataLoader(
    train_dataset,
    batch_size=config.training.batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=False,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config.training.batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
)

model = StockTransformer(
    input_dim=len(feature_columns),
    d_model=config.model.d_model,
    n_heads=config.model.n_heads,
    n_layers=config.model.n_layers,
    d_ff=config.model.d_ff,
    dropout=config.model.dropout,
    activation=config.model.activation,
    prediction_horizon=config.data.prediction_horizon,
)

trainer = Trainer(
    model=model,
    config=config,
    train_loader=train_loader,
    val_loader=val_loader,
)

Зареждане на локален dataset от: data\raw\sp500_stocks_data.parquet
Заредено! Размер: (1048575, 23)
Филтриране на данни... Първоначален размер: (1048575, 25)
Филтрирано! Финален размер: (10068, 25)


In [3]:
history = trainer.train()

Започване на обучение за 50 epochs...

Epoch 1/50... Train loss: 481783.140191 Val loss: 460049.044966 (178.8s)

Epoch 2/50... Train loss: 479725.319155 Val loss: 459027.645087 (180.2s)

Epoch 3/50... Train loss: 479101.341725 Val loss: 457966.093997 (177.8s)

Epoch 4/50... Train loss: 478524.838831 Val loss: 456843.429545 (164.2s)

Epoch 5/50... Train loss: 475283.117332 Val loss: 455683.677298 (168.4s)

Epoch 6/50... Train loss: 475567.144097 Val loss: 454495.485096 (176.2s)

Epoch 7/50... Train loss: 474436.197483 Val loss: 453296.377921 (166.2s)

Epoch 8/50... Train loss: 472481.404514 Val loss: 452070.473423 (162.9s)

Epoch 9/50... Train loss: 470869.399016 Val loss: 450831.648222 (168.2s)

Epoch 10/50... Train loss: 470960.336372 Val loss: 449582.889764 (173.5s)

Epoch 11/50... Train loss: 467657.965278 Val loss: 448324.802465 (162.6s)

Epoch 12/50... Train loss: 464789.914931 Val loss: 447066.494939 (165.5s)

Epoch 13/50... Train loss: 465751.325521 Val loss: 445805.345298 (165.